In [ ]:
import re
import json
import pdftotext
from refextract import extract_references_from_url
import hashlib
from pdf_process import cut_references, store_references, cut_references_clean
from pdf_process import metadata_pdf

: 

In [ ]:
def extract_content(pdf_id):
    path = "papers/extraction/{}.pdf".format(pdf_id)
    with open(path, "rb") as file:
        data = pdftotext.PDF(file)

    pdf_content = "\n".join(data)
    return pdf_content

def cut_references(pdf_url):
    # Extract references from pdf_url
    # using refextract library
    references = extract_references_from_url(pdf_url)

    # write references file 
    reffile = open("extracted_text/{}_raw.txt".format(pdf_url.split("/")[-1]), "a+")
    for ref in references:
        reffile.write(str(ref['raw_ref'])+'\n')

    reffile.close()

def cut_references_clean(pdf_url):
    # Cleant the repetitions
    output_file_path = "extracted_text/{}.txt".format(pdf_url.split("/")[-1])
    input_file_path = "extracted_text/{}_raw.txt".format(pdf_url.split("/")[-1])

    completed_lines_hash = set()

    output_file = open(output_file_path, "w")

    for line in open(input_file_path, "r"):

        hashValue = hashlib.md5(line.rstrip().encode('utf-8')).hexdigest()

        if hashValue not in completed_lines_hash:
            output_file.write(line)
            completed_lines_hash.add(hashValue)

    output_file.close()

In [ ]:
def cut_references(pdf_url):
    # Extract references from pdf_url
    # using refextract library
    references = extract_references_from_url(pdf_url)

    # write references file 
    reffile = open("extracted_text/{}_raw.txt".format(pdf_url.split("/")[-1]), "a+")
    for ref in references:
        reffile.write(str(ref['raw_ref'])+'\n')

    reffile.close()

In [ ]:
filename = 'fastapi/pdf_metadata.json'
pdf_id =  "2202.01752v1"

with open(filename,'r') as file:
    file_data = json.load(file)
    # Extract all the pdf dictionnaries
    papers = file_data['Papers']

    # find the pdf that we want
    pdf = metadata_pdf(pdf_id)

    url_pdf = pdf["url"]


    # Extract References from the pdf and store them in text file
    cut_references(url_pdf)
    cut_references_clean(url_pdf)
    print("PDF {}.pdf has been processed".format(pdf_id))
    
    references = []

In [ ]:
def extract_authors_from_apa_ref(apa_ref:str)->list:
    """Extract the authors from an APA reference
    Parameters:
    authors_string (str) : the string which might contain authors
    
    Returns:
    list: the list of authors as string elements
    """
    authors = []
    # Searching pattern 'T. (1995)' or ' T. (1995b)' like
    pattern = re.compile(r" [A-Z]\.\ \([1-2][0-9][0-9][0-9][a-z]*\)")
    # 3 = len(' T.'), which should be included in authors_string
    # try:
    end_authors_section_idx = re.search(pattern, apa_ref, flags=0).start() + 1
    # except AttributeError: # AttributeError: 'NoneType' object has no attribute 'start'
    #     # Searching pattern 'L. (Eds.). (1977)' like
    #     pattern = re.compile(r" [A-Z]\.\ \(Eds\.\) \([1-2][0-9][0-9][0-9][a-z]*\)")
    #     end_authors_section_idx = re.search(pattern, apa_ref, flags=0).start() + 1
    authors_string = apa_ref[0:end_authors_section_idx]
    authors_string = authors_string.replace('& ', '')
    authors_string = authors_string.replace('.,', '..,')
    authors = authors_string.split('., ')
    return authors